In [1]:
# リスト 2.1.17
# Webページのデータ読み込み（requests + BeautifulSoup を用いた最小例）
# ------------------------------------------------------------------------------------------------
# 目的:
#   - HTTP GET で HTML を取得し、BeautifulSoup で DOM をパースして解析の出発点となるオブジェクトを得る。
#
# 背景・理論メモ（コメントのみで挙動は変更しない）:
#   - HTTP クライアント: `requests.get` はデフォルトで TLS 検証あり（verify=True）。実運用では
#     タイムアウト（timeout=…）、再試行、適切な User-Agent の明示が推奨される。
#   - ステータス検査: 本スニペットでは最小例として status code を確認していない。堅牢化するなら
#     2xx 以外のハンドリング（例外/リトライ）を入れるのが一般的。
#   - 文字エンコーディング: `response.content` は「生バイト列」。`response.text` は
#     推定エンコーディングを通した文字列。Yahoo! JAPAN は UTF-8 だが、一般化するなら
#     `response.apparent_encoding` を使う選択肢もある（ここでは挙動を変えない）。
#   - パーサ選択: `"html.parser"` は標準ライブラリで依存が増えない一方、速度・寛容性は `lxml`
#     や `html5lib` に劣る場合がある。大規模クロールでは `lxml` がよく用いられる。
#   - 動的レンダリング: ニュースポータルはクライアント側で追加ロードされる要素（XHR/JS）もある。
#     本例は「初期 HTML の静的部分」だけが対象になる点に注意（Selenium/Playwright を使うと動的要素にも対応可）。
#   - 法的/倫理面: 利用規約・robots.txt・レート制御（sleep/Backoff）・アクセス頻度は順守すること。
#     研究/教育目的でも出力の再配布や大量取得には配慮が必要。
# ------------------------------------------------------------------------------------------------

import requests
from bs4 import BeautifulSoup

# Yahoo ニュースサイトのトップページ URL（静的 HTML の先頭部分のみ取得する最小例）
url = "https://news.yahoo.co.jp"

# HTTP GET を実行して HTML を取得
# - 本例では timeout やヘッダ（User-Agent）を省略している。実運用では追加を推奨。
# - 失敗時（例: 403, 429, 5xx）やネットワーク例外の捕捉は省略（最小例）。
html = requests.get(url)

# BeautifulSoup で HTML をパースして DOM ライクなオブジェクトへ変換
# - 第1引数に生バイト列（response.content）を渡すと、内部で自動デコードが行われる
# - パーサ "html.parser" は標準実装。速度重視なら "lxml" を選ぶことも多い（別途依存が必要）。
contents = BeautifulSoup(html.content, "html.parser")

In [2]:
# リスト 2.1.18
# サイトの情報確認（BeautifulSoup オブジェクトの直列化出力）
# --------------------------------------------------------------------------------
# 目的:
#   - 直前にパースした BeautifulSoup オブジェクト `contents` を、そのまま標準出力へ書き出して
#     取得済み HTML の構造や内容を目視確認する。
#
# 理論・背景（挙動は変えずに説明のみ）:
#   - print(contents) は、BeautifulSoup オブジェクトの __str__()（= 文字列化）を呼び出す。
#     これは「現在のパース木を HTML として再シリアライズした文字列」を返す。
#     ※ 重要: これは「元のレスポンスの生バイト列」ではなく、「パーサが正規化した後のマークアップ」。
#        つまり、属性順序や空白、欠落タグの自動補完などが起きるため、原文と完全一致はしない。
#   - パーサ依存性:
#       * "html.parser" は標準の寛容パーサで、軽量・依存少なめだが、`lxml` 等と比べて
#         正規化の結果やタグ修復の挙動が異なる。従って print(contents) の見た目は
#         パーサ選択に依存して変化し得る（決定性に影響）。
#   - 出力サイズと可観測性:
#       * トップページのような大規模 HTML をそのまま出力すると非常に長くなり、ログが読みにくい。
#         目視 EDA 用には `print(contents.prettify()[:2000])`（先頭のみ）や
#         `print(contents.title)`（<title> 要素のみ）などのプローブ出力が実務的。
#         ※ 本スニペットでは挙動を変えないため、そのまま全量出力にしている。
#   - DOM vs. レンダリング結果:
#       * ここで得ているのは「初期 HTML の静的部分」のパース木であり、ブラウザでの
#         追加レンダリング（JS による要素注入）結果ではない。動的に生成される
#         記事一覧等は含まれない可能性がある点に注意。
#   - 再現性と監査可能性:
#       * Yahoo!トップは時変データであり、取得時刻・User-Agent・地域設定により内容が変化する。
#         再現性が重要な場合、レスポンスのハッシュ（SHA-256）や取得時刻、HTTP ヘッダを
#         併せて保存するのが理にかなう（ここでは最小例のため省略）。
#
# 補足（代替出力例: 実装は変えずコメントのみ）:
#   - `print(contents.prettify())` : インデント付きで階層を把握しやすい
#   - `print(contents.title.string)` : タイトル文字列だけ確認
#   - `for a in contents.select('a'): print(a.get('href'))` : リンク一覧の抽出
#   これらは目的ごとに情報量を制御でき、ログ汚染を避けやすい。
#
# 注意:
#   - `contents` が None の場合は何も出力されない（通常は BeautifulSoup(...) が必ずオブジェクトを返す）。
#   - 文字化けがある場合は、requests 側のエンコーディング推定や `html = requests.get(...); html.encoding = 'utf-8'`
#     の明示、もしくは `BeautifulSoup(html.text, ...)` の利用を検討する。

print(contents)

<!DOCTYPE html>
<!DOCTYPE html>
<html lang="ja"><head><title>Yahoo!ニュース</title><meta content="text/html; charset=utf-8" http-equiv="Content-Type"/><meta content="width=1010" name="viewport"/><meta content="text/css" http-equiv="Content-Style-Type"/><meta content="text/javascript" http-equiv="Content-Script-Type"/><meta content="Yahoo!ニュースは、新聞・通信社が配信するニュースのほか、映像、雑誌や個人の書き手が執筆する記事など多種多様なニュースを掲載しています。" name="description"/><meta content="noarchive, max-image-preview:large" name="robots"/><meta content="telephone=no" name="format-detection"/><meta content="dVyTbsDzUrUAjCE1aCH9hKwZ8sOsCsvi2uhuBvasrp8" name="google-site-verification"/><meta content="Yahoo!ニュース" property="og:site_name"/><meta content="https://s.yimg.jp/images/news-web/versions/20251110-72f866c/all/images/ogp_default.png" property="og:image"/><meta content="Yahoo!ニュースは、新聞・通信社が配信するニュースのほか、映像、雑誌や個人の書き手が執筆する記事など多種多様なニュースを掲載しています。" property="og:description"/><meta content="Yahoo!ニュース" property="og:title"/><meta content="website" pro

In [3]:
# リスト2.1.19
# 必要な情報の抽出（Yahoo!ニュースのトップ HTML から記事タイトル候補を抽出して表示）
# --------------------------------------------------------------------------------
# 目的:
#   - BeautifulSoup の CSS セレクタ機能（.select）を用いて、特定のクラス名を持つ要素群を抽出し、
#     その可視テキストを標準出力へ列挙する。
#
# 背景・理論メモ:
#   - `.select(css_selector)` は CSS セレクタに一致する要素を **リスト** で返す（0件の場合は空リスト）。
#     セレクタ `.newsFeed_item_title` は「class='newsFeed_item_title' を持つ要素」を意味する。
#   - `.getText()` は `.get_text()` の別名で、要素配下の全テキストノードを連結して返す。
#     引数に `strip=True` 等を与えると前後空白を除去できるが、本スニペットでは **挙動を変えない** 方針のため未指定。
#   - 大規模サイトではクラス名が頻繁に変わる（A/Bテスト・デザイン刷新・難読化）ため、
#     セレクタが壊れて 0 件になる可能性がある点に注意。動的レンダリング要素は本手法では取得できないこともある。
#   - 可観測性と再現性の観点では、取得時の User-Agent・地域設定・取得時刻によって DOM が変わり得るため、
#     実運用ではメタ情報のログ化やフォールバックセレクタ（例: 見出しタグ 'h3 a' など）を併用すると堅牢性が上がる。
#   - 計算量は DOM ノード数を N とすると、おおまかに O(N) の走査。1ページ単位では現実的。

for title in contents.select(
    ".newsFeed_item_title"
):  # CSSクラス 'newsFeed_item_title' を持つ要素をすべて取得
    print(
        title.getText()
    )  # 要素配下のテキストを連結して出力（空白・改行は元HTML依存のまま）